# Initialize  (must have loaded MyCBR project via cmd)

In [1]:
from requests import get
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from IPython.display import HTML
from mycbr_py_api import MyCBRRestApi as mycbr
from pandas.io.json import json_normalize


# Wrappers and case base inits

In [6]:
import pyodbc
import pandas

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
server = 'localhost'
#server = 'hv-6066.idi.ntnu.no'

port = '8080'
base_url = 'http://' + server + ':' + port + '/'

obj = mycbr(base_url)
initCBCopy=obj.getAllCases();
currentCBCopy=obj.getAllCases();
#Retrieve the K most similar items to the case.    
def retrieve_c_att(case,k):
    raw = pd.DataFrame(requests.post(url=base_url + 'concepts/KP/casebases/CB_csvImport/amalgamationFunctions/default function/retrievalByMultipleAttributes?k='+str(k), json={'Municipality':case['Municipality'],'IndustrySubgroupCode':case['IndustrySubgroupCode'],'IndustryMainAreaCode':case['IndustryMainAreaCode'], 'ProbabilityForNonComplianceGivenControlPointandMunicipality':80,
    'ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup':80,'NumberofNonComplianceGivenControlPointandMunicipality':70,'NumberofNonComplianceGivenControlPointandIndustrySubgroup':70}).json()) 
    results = raw.apply(pd.to_numeric, errors='coerce').fillna(raw).sort_values(by='similarity', ascending=False)
    results['similarity'] = results['similarity'].astype(float)
    results['NonCompliance'] = results['NonCompliance'].astype(int)
    returned_df = results.apply(pd.to_numeric, errors='coerce').fillna(results)
    return results

def retrieve_c_att2(case,k):
    raw = pd.DataFrame(requests.post(url=base_url + 'concepts/KP/casebases/CB_csvImport/amalgamationFunctions/default function/retrievalByMultipleAttributes?k='+str(k), json={'IndustryAreaCode':case['IndustryAreaCode'],'IndustryGroupCode':case['IndustryGroupCode'],'IndustryCode':case['IndustryCode'],'MunicipalityNumber':case['MunicipalityNumber'],'IndustrySubgroupCode':case['IndustrySubgroupCode'],'IndustryMainAreaCode':case['IndustryMainAreaCode'], 'ProbabilityForNonComplianceGivenControlPointandMunicipality':100,
    'ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup':100,'NumberOfNonComplianceGivenControlPointandMunicipality':70,'NumberOfNonComplianceGivenControlPointandIndustrySubgroup':70, 'Fylke':case['Fylke']}).json())
    #print(case)
    results = raw.apply(pd.to_numeric, errors='coerce').fillna(raw).sort_values(by='similarity', ascending=False)
    results['similarity'] = results['similarity'].astype(float)
    results['NonCompliance'] = results['NonCompliance'].astype(int)
    returned_df = results.apply(pd.to_numeric, errors='coerce').fillna(results)
    return results
#Retrieve all items
def getAllCases():
    return obj.getAllCases()

#Add multiple contorl points to the case base
def addManyCases(cases):
    cases=cases.to_dict('records')
    rec={'cases':cases}
    #print(rec)
    #structure of case dictionary must be {'cases':[{'IndustrySubgroupCode':45.2},{"Municipality":"Amik"}]}
    #NB! int values must be formatted as strings
    results=requests.post(url=base_url + 'concepts/KP/casebases/CB_csvImport/cases', json=rec,headers={'Accept': 'application/json'})
    return results



#Fill casebase in case its empty
def fillCaseBase(splitindex):
    # Specifying the ODBC driver, server name, database, etc. directly
    cnxn = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER=localhost;DATABASE=IJCAI2022;Trusted_Connection=yes;')
    cursor = cnxn.cursor()

    traindateid=""
    testdateid=""
    cases=pd.read_sql("SELECT  [MuncipalityIndex] as MunicipalityIndex ,[IndustrySubgroupCodeIndex],[IndustryGroupCode],[IndustryAreaCode],[IndustryCode],[Fylke],[MunicipalityNumber],  [InspectionId] as 'InspectionId' ,[IndustryMainAreaCode],ControlPointText,[IndustrySubgroupCode], 'None' as UnderThemeName,[Municipality],[NonCompliance]"
+",round(isNull(SannsynlighetForBruddGittKPogKommune,(1.0/(6.0+isNull(tttg2,0))))*100,0) as ProbabilityForNonComplianceGivenControlPointandMunicipality"
+",round(isNull(SannsynlighetForBruddGittKPogNaring,(1.0/(6.0+isNull(tttg,0))))*100,0) as ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup"
+",round(isNull(AntBruddGittKPogNaring,0),0) as NumberOfNonComplianceGivenControlPointandIndustrySubgroup"
+",round(isNull(AntBruddGittKPogKommune,0),0) as NumberOfNonComplianceGivenControlPointandMunicipality"
                      +" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTest"+str(splitindex)+"]"
+" Left join(Select ( (Convert(real,isNull(Count(IndustrySubgroupCode),0))+1.0)/(AVG(Convert(real, Kommune3))+6.0))  as SannsynlighetForBruddGittKPogNaring,Count(IndustrySubgroupCode) as AntBruddGittKPogNaring,ControlPointText as KPT2, IndustrySubgroupCode as Nugk"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(splitindex)+"]"+" "  
+" join (Select Count(IndustrySubgroupCode) as Kommune3 , ControlPointText as KT2, IndustrySubgroupCode as Kom"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(splitindex)+"]"+" "+traindateid+"  group by ControlPointText, IndustrySubgroupCode) as a on a.Kom=IndustrySubgroupCode and ControlPointText=KT2" 
+" join (Select Count(Municipality) as Kommune5 ,  max(NonCompliance) as AntB2"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(splitindex)+"]"+" " +traindateid+") as c on c.AntB2=NonCompliance"
                    +traindateid
+" group by ControlPointText, NonCompliance, IndustrySubgroupCode) as b on b.KPT2=ControlPointText and b.Nugk=IndustrySubgroupCode"
+" Left join(Select ( (Convert(real,isNull(Count(Municipality),0))+1.0)/(AVG(Convert(real, Kommune3))+6.0))  as SannsynlighetForBruddGittKPogKommune,Count(Municipality) as AntBruddGittKPogKommune, ControlPointText as KPT, Municipality as KOMM"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(splitindex)+"]"+"  join (Select Count(Municipality) as Kommune3 , ControlPointText as KT2, Municipality as Kom"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(splitindex)+"]"+" "+traindateid+" group by ControlPointText, Municipality) as a on a.Kom=Municipality and ControlPointText=KT2" 
+" join (Select Count(Municipality) as Kommune5 ,  max(NonCompliance) as AntB2"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(splitindex)+"]"+" " +traindateid+") as c on c.AntB2=NonCompliance"
                    +traindateid+"  group by ControlPointText, NonCompliance, Municipality) as a on a.KOMM=Municipality and a.KPT=ControlPointText"
+" left join(Select Count(IndustrySubgroupCode) as tttg , ControlPointText as KT10, IndustrySubgroupCode as Kom10"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(splitindex)+"]"+" "+traindateid+" group by ControlPointText, IndustrySubgroupCode) as z on z.Kom10=IndustrySubgroupCode and ControlPointText=z.KT10" 
+" left join(Select Count(Municipality) as tttg2 , ControlPointText as KT10, Municipality as NUG10"
+" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(splitindex)+"]"+" "+traindateid+" group by ControlPointText, Municipality) as zz on ControlPointText=zz.KT10 and zz.NUG10=Municipality" 
+" where InspectionDateId<20190601"
                      +" Order by InspectionDateId " + testdateid,cnxn)
    #print(cases.head(5))
    
    cases['NonCompliance']= cases['NonCompliance'].map(str)
    cases['NumberOfNonComplianceGivenControlPointandIndustrySubgroup']= cases['NumberOfNonComplianceGivenControlPointandIndustrySubgroup'].map(str)
    cases['NumberOfNonComplianceGivenControlPointandMunicipality']= cases['NumberOfNonComplianceGivenControlPointandMunicipality'].map(str)
    cases['ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup']= cases['ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup'].astype(int)
    cases['ProbabilityForNonComplianceGivenControlPointandMunicipality']= cases['ProbabilityForNonComplianceGivenControlPointandMunicipality'].astype(int)
    cases['ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup']= cases['ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup'].map(str)
    cases['ProbabilityForNonComplianceGivenControlPointandMunicipality']= cases['ProbabilityForNonComplianceGivenControlPointandMunicipality'].map(str)
    cases['IndustrySubgroupCodeIndex']= cases['IndustrySubgroupCodeIndex'].map(str)
    cases['MunicipalityIndex']= cases['MunicipalityIndex'].map(str)
    cases['InspectionId']=cases['InspectionId'].apply(int)+0
    cases['InspectionId']= cases['InspectionId'].map(str)
    cases['MunicipalityNumber']= cases['MunicipalityNumber'].map(str)
    cases['IndustryAreaCode']= cases['IndustryAreaCode'].map(str)
    cases['IndustryCode']= cases['IndustryCode'].map(str)
    cases['IndustryCode']=cases['IndustryCode'].replace(to_replace=r',', value='.', regex=True)
    cases=cases.dropna()
   # print("Cases length:"+str(len(cases)))
    #print(cases['InspectionId'])
    return addManyCases(cases)
#Delete all items from the case base
def getCaseBaseIDs():

        final_url = base_url + '/casebases' 

        response = requests.get( url= final_url)
        casebases = response.json()

        return casebases
    
def addCaseBaseID (casebaseID):
    

        final_url = base_url + '/casebases/' + casebaseID       
        if casebaseID in getCaseBaseIDs():
            return -1

        response = requests.put( url= final_url)

        return response.json()
def deleteAllCases():
    final_urli = base_url + '/concepts/KP/casebases/CB_csvImport/cases/'
    response = requests.delete( url= final_urli)
    return response

#Updates casebase with the current set of cases (currentCBCopy)
def updateCaseBaseProb(cases):
    deleteAllCases()
    cases['NonCompliance']= cases['NonCompliance'].map(str)
    cases['NumberOfNonComplianceGivenControlPointandIndustrySubgroup']= cases['NumberOfNonComplianceGivenControlPointandIndustrySubgroup'].map(str)
    cases['NumberOfNonComplianceGivenControlPointandMunicipality']= cases['NumberOfNonComplianceGivenControlPointandMunicipality'].map(str)
    cases['ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup']= cases['ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup'].map(str)
    cases['ProbabilityForNonComplianceGivenControlPointandMunicipality']= cases['ProbabilityForNonComplianceGivenControlPointandMunicipality'].map(str)
    cases['IndustrySubgroupCodeIndex']= cases['IndustrySubgroupCodeIndex'].map(str)
    cases['MunicipalityIndex']= cases['MunicipalityIndex'].map(str)
    cases['InspectionId']=cases['InspectionId'].apply(int)+0
    cases['InspectionId']= cases['InspectionId'].map(str)
    cases['MunicipalityNumber']= cases['MunicipalityNumber'].map(str)
    cases['IndustryAreaCode']= cases['IndustryAreaCode'].map(str)
    cases['IndustryCode']= cases['IndustryCode'].map(str)
    cases['IndustryCode']=cases['IndustryCode'].replace(to_replace=r',', value='.', regex=True)

    cases=cases.drop(columns=[ 'caseID'])
    cases=cases.dropna()
    res=addManyCases(cases)
    return res;


#Updates casebase with the current set of cases (currentCBCopy)
def updateCaseBase():
    global currentCBCopy
    cases=currentCBCopy
    deleteAllCases()
    cases['NonCompliance']= cases['NonCompliance'].map(str)
    cases['NumberOfNonComplianceGivenControlPointandIndustrySubgroup']= cases['NumberOfNonComplianceGivenControlPointandIndustrySubgroup'].map(str)
    cases['NumberOfNonComplianceGivenControlPointandMunicipality']= cases['NumberOfNonComplianceGivenControlPointandMunicipality'].map(str)
    cases['ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup']= cases['ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup'].map(str)
    cases['ProbabilityForNonComplianceGivenControlPointandMunicipality']= cases['ProbabilityForNonComplianceGivenControlPointandMunicipality'].map(str)
    cases['IndustrySubgroupCodeIndex']= cases['IndustrySubgroupCodeIndex'].map(str)
    cases['MunicipalityIndex']= cases['MunicipalityIndex'].map(str)
    cases['InspectionId']=cases['InspectionId'].apply(int)+0
    cases['InspectionId']= cases['InspectionId'].map(str)
    cases['MunicipalityNumber']= cases['MunicipalityNumber'].map(str)
    cases['IndustryAreaCode']= cases['IndustryAreaCode'].map(str)
    cases['IndustryCode']= cases['IndustryCode'].map(str)
    cases['IndustryCode']=cases['IndustryCode'].replace(to_replace=r',', value='.', regex=True)

    cases=cases.drop(columns=[ 'caseID'])
    cases=cases.dropna()
    res=addManyCases(cases)
    return res;

#Restores the casebase back to it's original form (soft reset)
def restoreOriginalCaseBase():
    global initCBCopy
    cases=initCBCopy
    deleteAllCases()
    cases['NonCompliance']= cases['NonCompliance'].map(str)
    cases['NumberOfNonComplianceGivenControlPointandIndustrySubgroup']= cases['NumberOfNonComplianceGivenControlPointandIndustrySubgroup'].map(str)
    cases['NumberOfNonComplianceGivenControlPointandMunicipality']= cases['NumberOfNonComplianceGivenControlPointandMunicipality'].map(str)
    cases['ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup']= cases['ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup'].map(str)
    cases['ProbabilityForNonComplianceGivenControlPointandMunicipality']= cases['ProbabilityForNonComplianceGivenControlPointandMunicipality'].map(str)
    cases['IndustrySubgroupCodeIndex']= cases['IndustrySubgroupCodeIndex'].map(str)
    cases['MunicipalityIndex']= cases['MunicipalityIndex'].map(str)
    #cases['ï»¿EquityRatio']= cases['ï»¿EquityRatio'].map(str)
    #cases['InspectionId']= cases['InspectionId'].map(str)
    cases['InspectionId']=cases['InspectionId'].apply(int)+0
    cases['InspectionId']= cases['InspectionId'].map(str)
    cases['MunicipalityNumber']= cases['MunicipalityNumber'].map(str)
    cases['IndustryAreaCode']= cases['IndustryAreaCode'].map(str)
    cases['IndustryCode']= cases['IndustryCode'].map(str)
    cases['IndustryCode']=cases['IndustryCode'].replace(to_replace=r',', value='.', regex=True)

    cases=cases.drop(columns=[ 'caseID'])
    cases=cases.dropna()
    res=addManyCases(cases)
    return res;
#Restores the casebase back to it's original form (hard reset, slower)    
def restoreOriginalCaseBaseHard(iik):
    deleteAllCases()
    fillCaseBase(iik)
    
casebaseid='CB_csvImport'
if casebaseid not in getCaseBaseIDs():
    addCaseBaseID(casebaseid)

# Dynamic checklist methods

In [7]:
import math
import pyodbc
import pandas
import warnings
warnings.filterwarnings('ignore')
#An overview of industry codes in english can be in the appendix or at https://www.ssb.no/en/klass/klassifikasjoner/6 

k=50
r=0
currentChecklistIndex=0
interval=5 #How often recommendation will happen
nextrecommendation=interval
checklistInitLenght=15
currentChecklistIndex=0
parameters=list()
parametersmun=list()
currentCBCopy=initCBCopy
currentChecklistToPrint=pd.DataFrame(data={})
currentChecklist=pd.DataFrame(data={})
cases=obj.getAllCases().drop_duplicates(subset = ["InspectionId"])
cases2=obj.getAllCases()
    

recChecklist=pd.DataFrame(data={})
recChecklistSolution=pd.DataFrame(data={})
orglastindex=0
evaluatedChecklist=False
recChecklistPrevCP=list()
recChecklistPrevCPmun=list()
labellist=list()
labellistrecommanded=list()
rightlabellist=list()
evaluatedChecklistlabel=False

#This method is used to initialize and reset everything. Used when generating a new checklist for a new inspection
def initialize(iik):
    global currentChecklistIndex
    global currentCBCopy
    global initCBCopy
    global cases
    global cases2
    global currentChecklistToPrint
    global currentChecklist
    global orglastindex
    global inspection
    global interval #How often recommendation will happen
    global nextrecommendation 
    global checklistInitLenght
    global currentChecklistIndex
    global evaluatedChecklist
    global recChecklist
    global recChecklistPrevCP
    global recChecklistPrevCPmun
    global recChecklistSolution
    global labellist
    global parameters
    global parametersmun
    global labellistrecommanded
    global rightlabellist
    k=50
    r=0
    currentChecklistIndex=0
    nextrecommendation=interval
    parameters=list()
    parametersmun=list()
    currentChecklistToPrint=pd.DataFrame(data={})
    currentChecklist=pd.DataFrame(data={})

    if len(cases2)==0:
        initCBCopy=cases2

    if len(initCBCopy)==0:
        fillCaseBase(iik)
        initCBCopy=obj.getAllCases()
        currentCBCopy=initCBCopy
        cases=initCBCopy.drop_duplicates(subset = ["InspectionId"])
        cases2=initCBCopy
    if evaluatedChecklist:

        restoreOriginalCaseBase()
        currentCBCopy=initCBCopy
        cases=initCBCopy.drop_duplicates(subset = ["InspectionId"])
        cases2=initCBCopy

    recChecklist=pd.DataFrame(data={})
    recChecklistSolution=pd.DataFrame(data={})
    orglastindex=0
    recChecklistPrevCP=list()
    recChecklistPrevCPmun=list()
    evaluatedChecklist=False

#This method is used to get the input fields.
def getInspection():
    global Fylke
    global municipalityNumber
    global industrysubgroupcode
    global industrymainareacode
    
    
    inspection= {'Fylke': Fylke.get(),'Municipality':my_entry.get(),'MunicipalityNumber':int(municipalityNumber.get()), 'IndustrySubgroupCode':float(industrysubgroupcode.get()), 'IndustryMainAreaCode':industrymainareacode.get(),'IndustryAreaCode':int(str(industrysubgroupcode.get())[0:2]) 
                 ,'IndustryCode':float(str(industrysubgroupcode.get())[0:4]),'IndustryGroupCode':float(str(industrysubgroupcode.get())[0:5])  }    
    inspection['Fylke']=inspection['Fylke'].replace('ø','Ã¸')
    return inspection

#This method is used to build the initial checklist
def retrievechecklist(inspection2,iik):

    global currentChecklistIndex
    global currentCBCopy
    global cases
    global cases2
    global currentChecklistToPrint
    global currentChecklist
    global orglastindex
    global inspection
    global interval #How often recommendation will happen
    global nextrecommendation
    global checklistInitLenght
    global currentChecklistIndex
    global evaluatedChecklist
    
    initialize(iik)
    inspection=inspection2
    nextrecommendation=interval
    k=500
    r=0
    while r==0:
        cases=retrieve_c_att2(inspection,k)#Retrieve k cases (items) that matches inspection (query)
        uniqueCases=cases.drop_duplicates(subset = ["ControlPointText"])#Find all unique items by removing duplicate items


        if len(uniqueCases)>(checklistInitLenght-1):#Check if there are at least 10 unique cases
            r=1#Stop while loop when 10 unique items are found

            currentChecklist=uniqueCases.head(checklistInitLenght)        
            currentChecklist=currentChecklist.sort_values(by=['NumberOfNonComplianceGivenControlPointandIndustrySubgroup'], ascending=False)
            currentChecklist=currentChecklist.reset_index()
            orglastindex=currentChecklist["similarity"].iat[-1]


        k+=500 #Increment K while there are less then 10 unique items
    currentChecklistToPrint=currentChecklist.copy()
    currentChecklistToPrint["NonCompliance"]=currentChecklistToPrint["NonCompliance"].astype(float)
    currentChecklistToPrint["ControlPointText"]=currentChecklistToPrint["ControlPointText"].replace(to_replace=r'"', value='', regex=True)
    
    initlength=len(currentChecklist)
    if evaluatedChecklist==False:
        
        for ind, currentcp in currentChecklist.iterrows():
                    #Calculate the frequency of non-compliance given the item and industry subgroup code(specified in the inspection above) in the validation database
                    #Counting non-compliance for evaluation. First attempting to find non-compliance for the industry and municipality in the given inspection
            noncompliance=0
            lencurrentcc=0
            currentcc=cases2[(cases2["ControlPointText"]==currentcp["ControlPointText"]) ]
            currentcc=currentcc[(currentcc["IndustrySubgroupCode"].astype(float)==float(inspection["IndustrySubgroupCode"])) ]
            currentcc2=currentcc[(currentcc["MunicipalityNumber"].astype(int)==int(inspection["MunicipalityNumber"]))] 
            if len(currentcc2)!=0:
                lencurrentcc=len(currentcc2["NonCompliance"])
                noncompliance=len(currentcc2[currentcc2["NonCompliance"].astype(int)>0])
                currentChecklistToPrint.at[ind,"NonCompliance"]=noncompliance/lencurrentcc
            
                #If there are no records of the combination of item and the given inspection industry in validation data, then take the industry given in the item
            if len(currentcc2)==0:
                currentcc=cases2[(cases2["IndustrySubgroupCodeIndex"].astype(int)==int(currentcp["IndustrySubgroupCodeIndex"])) ]
                currentcc2=currentcc[(currentcc["MunicipalityIndex"].astype(int)==int(currentcp["MunicipalityIndex"])) ]
                lencurrentcc=len(currentcc2["NonCompliance"])
                noncompliance=len(currentcc2[currentcc2["NonCompliance"].astype(int)>0])
                currentChecklistToPrint.at[ind,"NonCompliance"]=noncompliance/lencurrentcc
            evaluatedChecklist=True

            #Code below is unreachable    
            
#This method is used to apply an item and automatically update the checklist with new items. Each time 5 items on the checklist have been answered, a dynamic update occurs.
def applyControlPoint(iik):
    global currentChecklistIndex
    global currentCBCopy
    global cases
    global cases2
    global currentChecklistToPrint
    global currentChecklist
    global orglastindex
    global inspection
    global interval #How often recommendation will happen
    global nextrecommendation 
    global checklistInitLenght
    global currentChecklistIndex
    global evaluatedChecklist
    global recChecklist
    global recChecklistPrevCP
    global recChecklistPrevCPmun
    global recChecklistSolution
    global parameters
    global parametersmun

    if currentChecklistIndex<len(currentChecklist):
        #drawing an answer to the current checklist item from a bernoulli distribution.
        currentChecklist.at[currentChecklistIndex,"NonCompliance"]=int(np.random.binomial(1,currentChecklistToPrint.at[currentChecklistIndex,"NonCompliance"]))


        # Specifying the ODBC driver, server name, database, etc. directly
        cnxn = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER=localhost;DATABASE=IJCAI2022;Trusted_Connection=yes;')
        cursor = cnxn.cursor()
        controlpointtext="'"+currentChecklist.at[currentChecklistIndex,"ControlPointText"]+"'"#"'Har virksomheten valgt verneombud?'"
        noncomplianceq=currentChecklist.at[currentChecklistIndex,"NonCompliance"]

        updatedprob=pandas.read_sql("Select 'None' as UnderThemeName, [MuncipalityIndex] as MunicipalityIndex,[IndustrySubgroupCodeIndex], [IndustryGroupCode],[IndustryAreaCode],[IndustryCode],[Fylke],[MunicipalityNumber],  InspectionId as InspectionId, ControlPointText,IndustryMainAreaCode,IndustrySubgroupCode,"
        +"Municipality,NonCompliance,[ProbabilityForNonComplianceGivenControlPointandMunicipality]"
        +",[ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup],[NumberOfNonComplianceGivenControlPointandIndustrySubgroup]"
        +",[NumberOfNonComplianceGivenControlPointandMunicipality],[IndustryPar1],[IndustryParTot],IndustryDynamicPar1,IndustryDynamicParTot"
        +",MunicipalityDynamicPar1,MunicipalityDynamicParTot,[MunicipalityPar1],[MunicipalityParTot] FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTestParameters"+str(iik)+"]"
        +" left join(SELECT [ControlPointText] as KPTKPT,KPT as Kontrollpunkttekst2,Municipality as KomKom,sum(iif(ANTB=-1,0,ANTB)) MunicipalityDynamicPar1"
        +",count(ANTB) MunicipalityDynamicParTot,CAST(sum(iif(ANTB=-1,0,ANTB)) as float)/count(ANTB) SannsynlighetForBruddGittTidligereKPKommuneogBrudd"
        +" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(iik)+"] join (select InspectionId as TiD, ControlPointText as KPT, NonCompliance as AntB from [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(iik)+"]"
        +") as a on a.TiD=InspectionId where ControlPointText="+controlpointtext
        +" and NonCompliance="+str(noncomplianceq)
        +"group by ControlPointText, KPT,Municipality) as c on c.Kontrollpunkttekst2=ControlPointText and c.KomKom=Municipality"
        +" left join(SELECT [ControlPointText] as KPTKPT2,KPT as Kontrollpunkttekst3,IndustrySubgroupCode as Nar2,"
        +"sum(iif(ANTB=-1,0,ANTB)) IndustryDynamicPar1,count(ANTB) IndustryDynamicParTot,CAST(sum(iif(ANTB=-1,0,ANTB)) as float)/count(ANTB) SannsynlighetForBruddGittTidligereKPNaringogBrudd"
        +" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(iik)+"] join (select InspectionId as TiD, ControlPointText as KPT, NonCompliance as AntB from [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(iik)+"])"
        +" as a on a.TiD=InspectionId where ControlPointText="+controlpointtext
        +" and NonCompliance="+str(noncomplianceq)
        +" group by ControlPointText, KPT,IndustrySubgroupCode) as cc on cc.Kontrollpunkttekst3=ControlPointText and cc.Nar2=IndustrySubgroupCode"
        +" join(select InspectionId as ttid FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTest"+str(iik)+"] where ControlPointText="
        +controlpointtext+" and NonCompliance="+str(noncomplianceq)+" group by InspectionId) as bb on bb.ttid=InspectionId"
        +" order by Kontrollpunkttekst2 asc"
                                    ,cnxn)
        
        #SQL Query retrieves existing and dynamic parameters for items conditioned on a answered item, the item outcome (answer),
        #Municipality (given) or industrysubgroupcode (given) and noncompliance (last are partitioned for the parameters *DynamicPar1 )
        updatedprob = updatedprob[~(updatedprob.IndustryDynamicParTot.isna()) | ~(updatedprob.MunicipalityDynamicParTot.isna())]

        #Dataframe with updated parameters
        currentCBCopyTest=currentCBCopy.copy()
        currentCBCopyTest['ControlPointText'] = currentCBCopyTest['ControlPointText'].str.replace(';','')

        updatedprob['MunicipalityPar1'] = updatedprob['MunicipalityPar1'].fillna(0)
        updatedprob['MunicipalityParTot'] = updatedprob['MunicipalityParTot'].fillna(0)
        updatedprob['MunicipalityDynamicPar1'] = updatedprob['MunicipalityDynamicPar1'].fillna(0)
        updatedprob['MunicipalityDynamicParTot'] = updatedprob['MunicipalityDynamicParTot'].fillna(0)
        updatedprob['IndustryPar1'] = updatedprob['IndustryPar1'].fillna(0)
        updatedprob['IndustryParTot'] = updatedprob['IndustryParTot'].fillna(0)
        updatedprob['IndustryDynamicPar1'] = updatedprob['IndustryDynamicPar1'].fillna(0)
        updatedprob['IndustryDynamicParTot'] = updatedprob['IndustryDynamicParTot'].fillna(0)


        updatedprob=updatedprob[updatedprob["IndustryMainAreaCode"]==inspection["IndustryMainAreaCode"]]

        
        updatedprobunique=updatedprob.drop_duplicates(subset = ["ControlPointText","IndustrySubgroupCode"])#,ignore_index=True update first
        updatedprobunique=updatedprobunique[updatedprobunique["IndustryMainAreaCode"]==inspection["IndustryMainAreaCode"]]
        updatedprobunique=updatedprobunique.reset_index()
        updatedprobunique["IndustrySubgroupCodeIndex"]=updatedprobunique["IndustrySubgroupCodeIndex"].astype(int)
        updatedprobunique["MunicipalityIndex"]=updatedprobunique["MunicipalityIndex"].astype(int)
        parameters.append(updatedprobunique.to_numpy())

        ccbii=currentCBCopyTest["IndustrySubgroupCodeIndex"].to_numpy()
        ccbii=ccbii.astype(int)
        ccbv=currentCBCopyTest["ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup"].to_numpy()
        ccbv=ccbv.astype(int)
        ccbi=currentCBCopyTest["IndustrySubgroupCode"].to_numpy()
        ccbi=ccbi.astype(float)
        ccbt=currentCBCopyTest["ControlPointText"].to_numpy()
        currentCBCopyTest=currentCBCopyTest.to_numpy()
        probForNonCompliance=currentCBCopy.columns.get_loc("ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup")
        industrySubgroupCodeIndexupdated=updatedprobunique.columns.get_loc("IndustrySubgroupCodeIndex")
        industryDynamicPar1=updatedprobunique.columns.get_loc("IndustryDynamicPar1")
        industryDynamicParTot=updatedprobunique.columns.get_loc("IndustryDynamicParTot")
        industryPar1=updatedprobunique.columns.get_loc("IndustryPar1")
        industryParTot=updatedprobunique.columns.get_loc("IndustryParTot")
        updatedprobunique=updatedprobunique.to_numpy()

        for case in updatedprobunique:
            caseIndex=int(case[industrySubgroupCodeIndexupdated])
            ccbiiBoolean=(ccbii==caseIndex)
            if len(parameters)>0:
                case[industryDynamicPar1]=0
                case[industryDynamicParTot]=0
                for indd in parameters:
                    inddind=indd[:,industrySubgroupCodeIndexupdated]
                    inddd=indd[(inddind==caseIndex)]
                    if len(inddd)>0:
                        case[industryDynamicPar1]=int(case[industryDynamicPar1])+int(inddd[0,industryDynamicPar1])
                        case[industryDynamicParTot]=int(case[industryDynamicParTot])+int(inddd[0,industryDynamicParTot])


            if len(parameters)>0 and case[industryDynamicParTot]>0:#avoid division by 0. case["IndustryDynamicParTot"]>29:
                value=((case[industryDynamicPar1])+1)/(case[industryDynamicParTot]+6)
            else:
                value=(case[industryPar1]+(case[industryDynamicPar1]))/(case[industryParTot]+(case[industryDynamicParTot]))
            ccbvtemp=ccbv[(ccbiiBoolean)]                    
            if len(ccbvtemp)>0 and int(round(value*100))!=ccbvtemp[0]:
                #currentCBCopyTest.at[ind, 'ProbabilityForNonComplianceGivenControlPointandMunicipality']=int(round(value.iat[0]*100))
                #Standard vector assignment in python is extremely slow so we have optimized it below
                ccase=currentCBCopyTest[(ccbiiBoolean)]
                ccase[:,probForNonCompliance]=int(round(value*100))
                currentCBCopyTest[(ccbiiBoolean)]=ccase

        currentCBCopyTest=pd.DataFrame(data=currentCBCopyTest[0:,0:],columns=currentCBCopy.columns)


        updatedprobunique=updatedprob.drop_duplicates(subset = ["ControlPointText","MunicipalityNumber"])#,ignore_index=True update pd first
        updatedprobunique=updatedprobunique.reset_index()
        updatedprobunique["IndustrySubgroupCodeIndex"]=updatedprobunique["IndustrySubgroupCodeIndex"].astype(int)
        updatedprobunique["MunicipalityIndex"]=updatedprobunique["MunicipalityIndex"].astype(int)

        parametersmun.append(updatedprobunique.to_numpy())
        if len(parametersmun)>30:
            parametersmun.pop(0)
        #print(str(len(parametersmun))+"\n")
        ccbii=currentCBCopyTest["MunicipalityIndex"].to_numpy()
        ccbii=ccbii.astype(int)
        ccbv=currentCBCopyTest["ProbabilityForNonComplianceGivenControlPointandMunicipality"].to_numpy()
        ccbv=ccbv.astype(int)
        ccbi=currentCBCopyTest["MunicipalityNumber"].to_numpy()
        ccbi=ccbi.astype(int)
        ccbt=currentCBCopyTest["ControlPointText"].to_numpy()
        probForNonCompliance=currentCBCopy.columns.get_loc("ProbabilityForNonComplianceGivenControlPointandMunicipality")
        municipalityIndexupdated=updatedprobunique.columns.get_loc("MunicipalityIndex")
        municipalityDynamicPar1=updatedprobunique.columns.get_loc("MunicipalityDynamicPar1")
        municipalityDynamicParTot=updatedprobunique.columns.get_loc("MunicipalityDynamicParTot")
        municipalityPar1=updatedprobunique.columns.get_loc("MunicipalityPar1")
        municipalityParTot=updatedprobunique.columns.get_loc("MunicipalityParTot")
        currentCBCopyTest=currentCBCopyTest.to_numpy()
        updatedprobunique=updatedprobunique.to_numpy()

        for case in updatedprobunique:
            caseIndex=int(case[municipalityIndexupdated])
            ccbiiBoolean=(ccbii==caseIndex)

            if len(parametersmun)>0:

                case[municipalityDynamicPar1]=0
                case[municipalityDynamicParTot]=0
                for indd in parametersmun:
                    inddind=indd[:,municipalityIndexupdated]
                    inddd=indd[(inddind==caseIndex)]
                    if len(inddd)>0:
                        case[municipalityDynamicPar1]=int(case[municipalityDynamicPar1])+int(inddd[0,municipalityDynamicPar1])
                        case[municipalityDynamicParTot]=int(case[municipalityDynamicParTot])+int(inddd[0,municipalityDynamicParTot])

            if len(parametersmun)>0 and case[municipalityDynamicParTot]>0:
                value=((case[municipalityDynamicPar1])+1)/(case[municipalityDynamicParTot]+6)
            else:
                value=(case[municipalityPar1]+(case[municipalityDynamicPar1]))/(case[municipalityParTot]+(case[municipalityDynamicParTot]))

            ccbvtemp=ccbv[(ccbiiBoolean)]
            if len(ccbvtemp)>0 and int(round(value*100))!=ccbvtemp[0]:#Optimisation: only update the values that have changed
                ccase=currentCBCopyTest[(ccbiiBoolean)]
                ccase[:,probForNonCompliance]=int(round(value*100))
                currentCBCopyTest[(ccbiiBoolean)]=ccase

        currentCBCopyTest=pd.DataFrame(data=currentCBCopyTest[0:,0:],columns=currentCBCopy.columns) 
        currentCBCopy=currentCBCopyTest
        
        #Check if the current checklist has recieved 5 answers since beginning or last recommendation.
        if len(parameters)>=nextrecommendation: #Update with recommendations after answering [nextrecommendation] items

            toUpdate=currentCBCopyTest[currentCBCopyTest["IndustryMainAreaCode"]==inspection["IndustryMainAreaCode"]]


            #Retrieve the updated cases for recommendation

            k=50
            r=0
            while r==0:
                cases=retrieve_c_att2(inspection,k)#Retrieve k cases (items) that matches inspection (query)
                uniqueCases=cases.drop_duplicates(subset = ["ControlPointText"])#Find all unique items by removing duplicate items
               # print(len(uniqueCases))
                # retain and remove/mark the controlled item. We are not applying the same item twice
                if len(uniqueCases)>(checklistInitLenght-1):#Check if there are at least 10 unique cases
                    r=1#Stop while loop when 10 unique items are found
                    #print(uniqueCases)#Print the current list of k cases
                    recChecklistPrevCP=list()
                    recChecklistPrevCPmun=list()
                    recChecklist=uniqueCases.head(checklistInitLenght)
                k+=400 #Increment K while there are less then 10 unique items

            for ind, case in currentChecklist.iterrows():
                recc=recChecklist["ControlPointText"].to_numpy()
                recChecklist=recChecklist[recc!=case["ControlPointText"]]

            #print("\nlast index: "+str(orglastindex))    
            recChecklist=recChecklist[recChecklist["similarity"]>= orglastindex]




            recChecklistSolution=recChecklist.copy()
            recChecklistSolution["NonCompliance"]=recChecklistSolution["NonCompliance"].astype(float)
            cases2industrysubgroupcodeindex=cases2.columns.get_loc("IndustrySubgroupCodeIndex")
            cases2municipalityindex=cases2.columns.get_loc("MunicipalityIndex")
            cases2inspectionid=cases2.columns.get_loc("InspectionId")
            cases2noncompliance=cases2.columns.get_loc("NonCompliance")
            cases2industrysubgroupcode=cases2.columns.get_loc("IndustrySubgroupCode")
            cases2controlpointtext=cases2.columns.get_loc("ControlPointText")
            cases2nu=cases2.to_numpy()
            for ind, currentcp in recChecklist.iterrows():
                inndex=0
                counter=0
                maxscore=0
                noncompliance=0
                lencurrentcc=0

                for indd in parameters:

                    inddd=indd[(indd[:,industrySubgroupCodeIndexupdated].astype(int)==int(currentcp["IndustrySubgroupCodeIndex"]))]
                    if len(inddd)>0:
                        if inddd[0,industryDynamicParTot]>maxscore:
                            maxscore=inddd[0,industryDynamicParTot]
                            parameter=indd
                            indddm=inddd
                            inndex=counter

                    counter=counter+1
                
                prevcp=currentChecklist.loc[inndex]
                cases22=cases2[(cases2nu[:,cases2industrysubgroupcodeindex].astype(int)==int(prevcp["IndustrySubgroupCodeIndex"]))]
                cases22prox=cases22["NonCompliance"].to_numpy()
                cases22=cases22[(cases22prox.astype(int)==int(prevcp["NonCompliance"]))] #only select inspections with matching municipality and same noncompliance status
                #print("Sub:"+str(prevcp))
                if maxscore>0:
                    recChecklistPrevCP.append(str(prevcp["ControlPointText"])+"")
                else:
                    recChecklistPrevCP.append(str(""))
                condcaseids=cases22.drop_duplicates(subset = ["InspectionId"])
                condcaseids=condcaseids["InspectionId"].to_numpy()
                for  cc in condcaseids:
                    currentcc=cases2nu[(cases2nu[:,cases2inspectionid].astype(int)==int(cc))]
                    currentcc=currentcc[(currentcc[:,cases2industrysubgroupcodeindex].astype(int)==int(currentcp["IndustrySubgroupCodeIndex"]))]
                    lencurrentcc=lencurrentcc+len(currentcc[:,cases2noncompliance])
                    noncompliance=noncompliance+len(currentcc[currentcc[:,cases2noncompliance].astype(int)>0])

                prevvalue=0
                if lencurrentcc>0:
                    recChecklistSolution.at[ind,"NonCompliance"]=float(noncompliance)/float(lencurrentcc)

                    prevvalue=1

                inndex=0
                counter=0
                maxscore=0
                noncompliance=0
                lencurrentcc=0
                for indd in parametersmun:

                    inddd=indd[(indd[:,municipalityIndexupdated]==currentcp["MunicipalityIndex"])]
                    if len(inddd)>0:
                        if inddd[:,municipalityIndexupdated]>maxscore:
                            maxscore=inddd[:,municipalityIndexupdated]#TODO: CHECK LOGICZ HERE
                            parameter=indd
                            indddm=inddd
                            inndex=counter
                    counter=counter+1

                prevcp=currentChecklist.loc[inndex]
                cases22=cases2[(cases2nu[:,cases2municipalityindex].astype(int)==int(prevcp["MunicipalityIndex"]))]
                cases22prox=cases22["NonCompliance"].to_numpy()
                cases22=cases22[(cases22prox.astype(int)==int(prevcp["NonCompliance"]))] #only select inspections with matching municipality and same noncompliance status

                if maxscore>0:

                    recChecklistPrevCPmun.append(str(prevcp["ControlPointText"])+"")
                else:
                    recChecklistPrevCPmun.append(str(""))

                condcaseids=cases22.drop_duplicates(subset = ["InspectionId"])
                condcaseids=condcaseids["InspectionId"].to_numpy()
                for cc in condcaseids:
                    currentcc=cases2nu[(cases2nu[:,cases2inspectionid].astype(int)==int(cc)) ]
                    currentcc=currentcc[(currentcc[:,cases2municipalityindex].astype(int)==int(currentcp["MunicipalityIndex"]))]
                    lencurrentcc=lencurrentcc+len(currentcc[:,cases2noncompliance])
                    noncompliance=noncompliance+len(currentcc[currentcc[:,cases2noncompliance].astype(int)>0])

                if lencurrentcc>0:

                    if prevvalue==1:
                        recChecklistSolution.at[ind,"NonCompliance"]=(float(recChecklistSolution.at[ind,"NonCompliance"])+float(noncompliance)/float(lencurrentcc))/2      
                    else:
                        recChecklistSolution.at[ind,"NonCompliance"]=float(noncompliance)/float(lencurrentcc)

                    
                if lencurrentcc==0 and prevvalue==0:#If no conditional non-compliance exists in validationbase
                    noncompliance=0
                    lencurrentcc=0
                    currentcc=cases2nu[(cases2nu[:,cases2industrysubgroupcode].astype(float)==float(inspection["IndustrySubgroupCode"])) & (cases2nu[:,cases2municipalityindex]==currentcp["MunicipalityIndex"]) ]

                    if len(currentcc[:,0])==0:

                        currentcc=cases2nu[(cases2nu[:,cases2industrysubgroupcode].astype(float)==float(inspection["IndustrySubgroupCode"])) & (cases2nu[:,cases2controlpointtext]==currentcp["ControlPointText"]) ]
                            #If there are no records of the combination of item and the given inspection industry in validation data, then take the industry given in the item
                    if len(currentcc[:,0])==0:#Last resort

                        currentcc=cases2nu[(cases2nu[:,cases2industrysubgroupcodeindex].astype(int)==int(currentcp["IndustrySubgroupCodeIndex"])) ]
                    lencurrentcc=lencurrentcc+len(currentcc[:,cases2noncompliance])
                    noncompliance=noncompliance+len(currentcc[currentcc[:,cases2noncompliance].astype(int)>0])
                    noncompliancefrac=0
                    if lencurrentcc>0:
                        noncompliancefrac=(noncompliance/lencurrentcc)

                    recChecklistSolution.at[ind,"NonCompliance"]=noncompliancefrac
                    


            if len(recChecklist)>0:
                recChecklistSolution["ControlPointText"]=recChecklistSolution["ControlPointText"].replace(to_replace=r'"', value='', regex=True)

                addRecommandedControlPoints()
                print(len(recChecklistPrevCP))
                print(len(recChecklistPrevCPmun))
                i=0
                for cp in recChecklistSolution["ControlPointText"]:
                    recChecklistPrevCP[i]=recChecklistPrevCP[i].replace('"', '')
                    recChecklistPrevCPmun[i]=recChecklistPrevCPmun[i].replace('"', '')
                    i=i+1
                #print(currentChecklistToPrint["ControlPointText"])
                
            nextrecommendation=nextrecommendation+interval
        currentChecklistIndex=currentChecklistIndex+1
    if currentChecklistIndex>=len(currentChecklist):

        print("\n The simulated inspection has been completed with a total of "+ str(int(round(currentChecklist["NonCompliance"].sum())))+" noncompliances found.")

    if currentChecklistIndex<len(currentChecklist):
        return pd.DataFrame(data={})
    else:
        return currentChecklistToPrint

#This method appends the recommended items to the checklist
def addRecommandedControlPoints():
    global currentChecklist
    global currentChecklistToPrint
    global recChecklist
    global recChecklistSolution
    global recChecklistPrevCP
    global recChecklistPrevCPmun
    currentChecklist=currentChecklist.append(recChecklist,ignore_index = True)
    currentChecklistToPrint=currentChecklistToPrint.append(recChecklistSolution,ignore_index = True)
    
#This method simulates inspection and returns the checklist. Note that only the added/recommanded items have updated
#probability estimates according to the context of the inspection
def simulateInspection(inspection2,iik):
    
    retrievechecklist(inspection2,iik)
    r=0
    check=0
    while r==0:
        check+=1
        if check==100:
            break
        simulatedChecklist=applyControlPoint(iik)
        if len(simulatedChecklist)>0:
            return simulatedChecklist
        

# Checklist  experiment

In [14]:
#inspection= {'Municipality':'Bergen', 'IndustrySubgroupCode':47.111,'IndustryMainAreaCode':'G' } shows the system doesn't recommand further CP if everything is in order
import time
import pickle
interval=5 #How often recommendation will happen
for ik in range(0,8):
    deleteAllCases()
    fh2=open("Log_" + "ContextAwareCBRNBIChecklistsTEST"+str(ik) + '.txt', 'w+')
    if len(cases2)==0:
        initCBCopy=cases2
    if len(initCBCopy)==0:
        fillCaseBase(ik)
        initCBCopy=obj.getAllCases()
        currentCBCopy=initCBCopy
    else:
        restoreOriginalCaseBaseHard(ik)#Hard reset
        initCBCopy=obj.getAllCases()
        currentCBCopy=initCBCopy
    cases2=initCBCopy
    #print(len(initCBCopy))        
    start_time = time.time()
    inspectionstot=0
    checklistlen=0
    averagenon=0
    nextrecommendation=interval
    checklistInitLenght=15
    currentChecklistIndex=0
    parameters=list()
    parametersmun=list()
    currentCBCopy=initCBCopy
    precchecklists=0
    recchecklists=0
    accchecklists=0
    precisionoriginal=0
    cases=initCBCopy
    lengthprecgt=0.0000001
    inspectionCases=initCBCopy.drop_duplicates(subset = ["InspectionId"])
    inspectionCases=inspectionCases.sort_values(by=['InspectionId'], ascending=False,ignore_index=True)
    currentind=0
    try:
        with open('currentstate'+str(ik)+'.pkl', 'rb') as handle2:#Lets you resume execution if system crashes or similar
            currentstate = pickle.load(handle2)
    except (OSError, IOError) as e:
        currentstate=""
        #continue
    if currentstate!="":
       #Resuming the current state of the experiment
        currentind=currentstate['index']+1
        inspectionstot=currentstate['inspectionstot']
        precisionoriginal=currentstate['precisionoriginal']
        lengthprecgt=currentstate['lengthprecgt']
        checklistlen=currentstate['checklistlen']
        averagenon=currentstate['averagenon']
        precchecklists=currentstate['precchecklists']
        recchecklists=currentstate['recchecklists']
        accchecklists=currentstate['accchecklists']
        start_time=start_time-currentstate['timetotal']


    inspectionCases['Fylke']=inspectionCases['Fylke'].replace('ø','Ã¸')
    counter=0
    #Iterate through all the inspections in the current validation fold. A case corresponds to an inspection.
    for ind, case in inspectionCases.iloc[currentind:-1].iterrows():
        print("Simulating inspection for input validation case: "+str(case)) 
        counter+=1
        
        uniqueChecklist=simulateInspection(case,ik)#simulates an inspection and retrieve the resulting checklist (uniqueChecklist).
        inspectionstot+=1#len(uniqueChecklist)
        #Retrieve/prepare the original, ground truth checklist from the current inspection instance/case.
        originalChecklist=cases[cases["InspectionId"].astype(str)==str(case["InspectionId"])]
        originalChecklist["NonCompliance"]=originalChecklist["NonCompliance"].astype(float)
        originalChecklist["ControlPointText"]=originalChecklist["ControlPointText"].replace(to_replace=r'"', value='', regex=True)
        precpercl=0
        lengthorgcl=0
        #Calculate ground truth precision.
        for ind2, generatedChecklistControlpoint in uniqueChecklist.iterrows():
            excp=originalChecklist[originalChecklist["ControlPointText"]==generatedChecklistControlpoint["ControlPointText"]]

            if len(excp)>0:
                #calculate ground truth precision for if excp contains an checklist item.
                #To reduce the risk of calculation error we do a division here, to make sure that precpercl is not higher than 1.
                precpercl+=(excp["NonCompliance"].sum()/len(excp))
                lengthorgcl+=1
        #Calculate the ground truth precision for the current checklist : total number of positive items divided by checklist length
        if lengthorgcl>0:
            precisionoriginal+=(precpercl/lengthorgcl)
            lengthprecgt+=1
        #Perparing to calculate estimated prec, acc and rec.
        caseMatch=initCBCopy[initCBCopy["IndustrySubgroupCode"].astype(float)==float(case["IndustrySubgroupCode"])]
        caseMatch=caseMatch[caseMatch["MunicipalityNumber"].astype(int)==int(case["MunicipalityNumber"])]
        caseMatch["NonCompliance"]=caseMatch["NonCompliance"].astype(float)
        caseMatch["ControlPointText"]=caseMatch["ControlPointText"].replace(to_replace=r'"', value='', regex=True)
        negatives=caseMatch
        #Find all negatives by filtering out any cases with an item that exists on the generated checklist
        for ind2, generatedChecklistControlpoint in uniqueChecklist.iterrows():
            negatives=negatives[negatives["ControlPointText"]!=generatedChecklistControlpoint["ControlPointText"]]

        #Find the numbers of unique predicted negatives and estimate the numbers of true and false negatives.
        uniquenegatives=len(negatives.drop_duplicates(subset = ["ControlPointText"]))
        negativecpy=negatives.copy()
        negativecpy["NonCompliance"]=negativecpy["NonCompliance"].astype(float)
        groupedby=negativecpy.groupby(["ControlPointText"],as_index=False).count()
        groupedbys=negativecpy.groupby(["ControlPointText"],as_index=False).sum()
        groupedby["NonCompliance"]=groupedbys["NonCompliance"]/groupedby["NonCompliance"]
        uniquecalcnegatives=groupedby.drop_duplicates(subset = ["ControlPointText"])
        noncompliancenegatives=uniquecalcnegatives["NonCompliance"].sum()
        falsenegativeelement=0
        if uniquenegatives>0:
            falsenegativeelement=noncompliancenegatives/uniquenegatives #To avoid selection bias effects
        truenegativeelement=1-falsenegativeelement
        #update total checklist length and calculate numbers of true/false positives/negatives for the current checklist
        checklistlen+=len(uniqueChecklist)
        truepositivesprchecklist=uniqueChecklist["NonCompliance"].sum()
        falsepositivesprchecklist=(len(uniqueChecklist)-truepositivesprchecklist)
        truenegativesprchecklist=truenegativeelement*uniquenegatives
        falsenegativesprchecklist=falsenegativeelement*uniquenegatives
        
        #Then calculate and update precision, acc and recall score.
        averagenon+=truepositivesprchecklist
        
        precprchecklist=truepositivesprchecklist/len(uniqueChecklist)
        precchecklists+=precprchecklist

        recprchecklist=0
        if (truepositivesprchecklist+falsenegativesprchecklist)>0:
            recprchecklist=truepositivesprchecklist/(truepositivesprchecklist+falsenegativesprchecklist)
        recchecklists+=recprchecklist

        accprchecklist=0
        if (len(uniqueChecklist)+uniquenegatives)>0:
            accprchecklist=(truepositivesprchecklist+truenegativesprchecklist)/(len(uniqueChecklist)+uniquenegatives)
        accchecklists+=accprchecklist
        #Prints the current average precision, acc and recall scores as the experiment progresses (for the current validation fold).
        print("Current Average Precision is: "+str((precchecklists)/(inspectionstot)))
        print("Current Average Accuracy is: "+str((accchecklists)/(inspectionstot)))
        print("Current Average Recall is: "+str((recchecklists)/(inspectionstot)))

        currentstate={'index':ind, 'inspectionstot':inspectionstot,'precisionoriginal':precisionoriginal,'lengthprecgt':lengthprecgt,
                    'checklistlen':checklistlen,'averagenon':averagenon,'precchecklists':precchecklists,'recchecklists':recchecklists,
                     'accchecklists':accchecklists,'timetotal':(time.time()-start_time)}
        with open('currentstate'+str(ik)+'.pkl', 'wb') as handle:
            pickle.dump(currentstate, handle)

    #Calculate statistics per validation fold. After crossvalidation, the numbers below are manually averaged for all 8 test folds.
    print("\nTime total: "+str((time.time()-start_time)))
    fh2.write("\nTime total: "+str((time.time()-start_time)))
    print("Precision: "+str((precchecklists)/(inspectionstot)))
    fh2.write("\nPrecision: "+str((precchecklists)/(inspectionstot)))
    print("Precision Ground Truth: "+str((precisionoriginal)/(lengthprecgt)))
    fh2.write("\nPrecision Ground Truth: "+str((precisionoriginal)/(lengthprecgt)))
    print("Recall: "+str((recchecklists)/(inspectionstot)))
    fh2.write("\nRecall: "+str((recchecklists)/(inspectionstot)))
    print("Accuracy: "+str((accchecklists)/(inspectionstot)))
    fh2.write("\nAccuracy: "+str((accchecklists)/(inspectionstot)))
    print("Average number of items per checklist: "+str((checklistlen)/inspectionstot))
    fh2.write("\nAverage number of items per checklist: "+str((checklistlen)/inspectionstot))
    print("Average number of non-compliant items per checklist: "+str((averagenon)/inspectionstot))
    fh2.write("\nAverage number of non-compliant items per checklist: "+str((averagenon)/inspectionstot))
    fh2.close()


Simulating inspection for input validation case: caseID                                                             KP-CB_csvImport69623                                                                                                                                         
ControlPointText                                                   " Does the person responsible for the major accident activity ensure that the contingency plan is reviewed and tested to meet relevant emergency situations?"
Fylke                                                              Oslo                                                                                                                                                         
IndustryAreaCode                                                   46                                                                                                                                                           
IndustryCode                                       


 The simulated inspection has been completed with a total of 3 noncompliances found.
Current Average Precision is: 0.25
Current Average Accuracy is: 0.7548076923076923
Current Average Recall is: 0.9166666666666667
Simulating inspection for input validation case: caseID                                                             KP-CB_csvImport80147                                                                                                                                                                       
ControlPointText                                                   " Has the employer ensured that the employee has been given a written statement of the method of calculating salary, the calculation basis for holiday pay and deductions made (pay slip)?"
Fylke                                                              Rogaland                                                                                                                                                       


 The simulated inspection has been completed with a total of 11 noncompliances found.
Current Average Precision is: 0.47142857142857136
Current Average Accuracy is: 0.767806617538089
Current Average Recall is: 0.8098784488252317
Simulating inspection for input validation case: caseID                                                             KP-CB_csvImport120534                                                                                                                                                                      
ControlPointText                                                   " Has the employer ensured that the employee has been given a written statement of the method of calculating salary, the calculation basis for holiday pay and deductions made (pay slip)?"
Fylke                                                              Agder                                                                                                                                           


 The simulated inspection has been completed with a total of 7 noncompliances found.
Current Average Precision is: 0.44817460317460317
Current Average Accuracy is: 0.7729159126667181
Current Average Recall is: 0.7185563532708752
Simulating inspection for input validation case: caseID                                                             KP-CB_csvImport114164                                                                          
ControlPointText                                                   " Can the employer submit documentation of a periodic expert inspection of the work equipment?"
Fylke                                                              Viken                                                                                          
IndustryAreaCode                                                   49                                                                                             
IndustryCode                                                       49


 The simulated inspection has been completed with a total of 9 noncompliances found.
Current Average Precision is: 0.46977072310405643
Current Average Accuracy is: 0.7389933529401264
Current Average Recall is: 0.6881692075896564
Simulating inspection for input validation case: caseID                                                             KP-CB_csvImport35045                                                                                                                                                                                  
ControlPointText                                                   " Has the employer mapped the dangers and problems the employees may be exposed to during excavation work and on this basis assessed the risk of injury or danger to the employees health and safety?"
Fylke                                                              Viken                                                                                                                     

KeyboardInterrupt: 

# Restore original casebase

In [114]:
#print(restoreOriginalCaseBase())

<Response [200]>


In [5]:
if len(cases2)==0:
    initCBCopy=cases2
if len(initCBCopy)==0:
    fillCaseBase(0)
    initCBCopy=obj.getAllCases()
    currentCBCopy=initCBCopy
else:
    restoreOriginalCaseBaseHard(0)#Hard reset
    initCBCopy=obj.getAllCases()
    currentCBCopy=initCBCopy